# ETL (Extracción, Transformación y Carga) para el conjunto de datos

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

## Creación de los Datasets
Cargamos archivo Excel y procedemos a leer la información

In [2]:
# Cargando el archivo Excel
hechos_df = pd.read_excel('../Datasets/homicidios.xlsx', sheet_name= "HECHOS")
victimas_df = pd.read_excel('../Datasets/homicidios.xlsx', sheet_name= "VICTIMAS")

## Dataset Hechos
Visualizando las primeras filas de cada hoja para entender la estructura de los datos.

In [3]:
hechos_df.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [4]:
victimas_df.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


Verificando si hay valores faltantes en las hojas 'HECHOS' y 'VICTIMAS'

In [5]:
hechos_df.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

In [6]:
victimas_df.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

Realizando una revisión manual en el archivo Excel, se valida que algunos registros no tienen la información completa en las columnas 'XY (CABA)', 'pos x' y 'pos y', especialmente la columna 'XY (CABA)' que tiene información incompleta como 'Point (. .)'; se procede a separar los registros incompletos en un Dataframe para posteriormente completarlos.

In [7]:
sinXY_df = hechos_df[hechos_df['XY (CABA)']=='Point (. .)'][['ID','LUGAR_DEL_HECHO', 'Dirección Normalizada', 'XY (CABA)', 'pos x', 'pos y']]
sinXY_df

,ID,LUGAR_DEL_HECHO,Dirección Normalizada,XY (CABA),pos x,pos y
35,2016-0049,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),-58.37714647568196,-34.63657525428238
38,2016-0052,AUTOPISTA LUGONES PK 10000,NaN,Point (. .),.,.
71,2016-0096,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,Point (. .),-58.47433193007387,-34.66684950051973
106,2016-0136,AU BUENOS AIRES - LA PLATA KM. 4,NaN,Point (. .),.,.
119,2016-0151,SD,NaN,Point (. .),.,.
139,2016-0174,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,Point (. .),.,.
176,2017-0042,AV. LEOPOLDO LUGONES PKM 6900,"LUGONES, LEOPOLDO AV.",Point (. .),.,.
180,2017-0050,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,NaN,Point (. .),.,.
181,2017-0051,AU DELLEPIANE 2400,NaN,Point (. .),.,.
256,2017-0140,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),.,.


### Limpieza de datos
Utilizando la libreria `geopy` con su clase `Nominatim` se completará las coordenadas faltantes.

In [8]:
geolocalizador = Nominatim(user_agent="myGeocoder", timeout=20)

# Creamos la función geolocalizar que se utilizará para enviar la dirección y retornar la geolocalización
def geolocalizar(address):
     try:
          if 'Buenos Aires, Argentina' not in address:
               address += ', Buenos Aires, Argentina'
          location = geolocalizador.geocode(address)
          return (location.latitude, location.longitude)
     except Exception as e:
          return (None, None)

Agregamos una columna 'coordenadas' a sinXY_df, convirtiendo direcciones en 'LUGAR_DEL_HECHO' a coordenadas geográficas mediante la función geolocalizar, y asigna None en caso de falla en la geolocalización.

In [9]:
sinXY_df['coordenadas'] =sinXY_df['LUGAR_DEL_HECHO'].apply(geolocalizar)
sinXY_df

,ID,LUGAR_DEL_HECHO,Dirección Normalizada,XY (CABA),pos x,pos y,coordenadas
35,2016-0049,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),-58.37714647568196,-34.63657525428238,"(None, None)"
38,2016-0052,AUTOPISTA LUGONES PK 10000,NaN,Point (. .),.,.,"(None, None)"
71,2016-0096,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,Point (. .),-58.47433193007387,-34.66684950051973,"(None, None)"
106,2016-0136,AU BUENOS AIRES - LA PLATA KM. 4,NaN,Point (. .),.,.,"(None, None)"
119,2016-0151,SD,NaN,Point (. .),.,.,"(None, None)"
139,2016-0174,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,Point (. .),.,.,"(None, None)"
176,2017-0042,AV. LEOPOLDO LUGONES PKM 6900,"LUGONES, LEOPOLDO AV.",Point (. .),.,.,"(None, None)"
180,2017-0050,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,NaN,Point (. .),.,.,"(None, None)"
181,2017-0051,AU DELLEPIANE 2400,NaN,Point (. .),.,.,"(None, None)"
256,2017-0140,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),.,.,"(None, None)"


Dado que son pocos datos, se realiza la actualización manualmente para los ID: 139, 181 Y 559

In [10]:
# Lista de IDs para geolocalizar
ids_a_geolocalizar = [139, 181, 559]

# Iterando sobre los IDs
for id in ids_a_geolocalizar:
    if id in hechos_df.index:
        lat, lon = geolocalizar(hechos_df.at[id, 'LUGAR_DEL_HECHO'])
        hechos_df.at[id, 'pos y'] = lat
        hechos_df.at[id, 'pos x'] = lon

Con la finalidad de terminar de completar los datos vacíos, se reemplazará para la columna 'XY (CABA)' los valores 'Point (. .)' por 0, en las columnas 'pos x' y 'pos y' los '.' por 0

In [11]:
hechos_df['pos x'] = hechos_df['pos x'].replace('.', 0)
hechos_df['pos y'] = hechos_df['pos y'].replace('.', 0)
hechos_df['XY (CABA)'] = hechos_df['XY (CABA)'].replace('Point (. .)', 0)

Columnas "Cruce" y "Altura"
La columna "Cruce" indica si el accidente ocurrió en una intersección de vías, mientras que la columna "Altura" indica si el accidente ocurrio en mitad de una vía. Con la finalidad de reducir la cantidad de campos vacíos, se procede a trabajar con la columna "Cruce", para los registros que los accidentes fueron en un cruce el valor será `True` y para los casos que los accidentes ocurrieron en una altura el valor será `False`

In [12]:
hechos_df.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [13]:
hechos_df = hechos_df.drop('Altura', axis=1)

In [14]:
hechos_df['Cruce'] = hechos_df['Cruce'].notna()
hechos_df.head(20)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,True,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,False,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,True,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,True,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
5,2016-0008,1,2016-01-24,2016,1,24,18:30:00,18,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,27 DE FEBRERO AV.,True,27 DE FEBRERO AV. y ESCALADA AV.,8,Point (101721.59002217 93844.25656649),-58.44451316,-34.68475866,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
6,2016-0009,1,2016-01-24,2016,1,24,19:10:00,19,NOGOYA Y JOAQUIN V. GONZALES,CALLE,NOGOYA,True,"NOGOYA y GONZALEZ, JOAQUIN V.",11,Point (96545.87592078 102330.67262199),-58.50095869,-34.60825440,MOTO-AUTO,MOTO,AUTO
7,2016-0010,1,2016-01-29,2016,1,29,15:20:00,15,AV GENERAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,MOTO-AUTO,MOTO,AUTO
8,2016-0012,1,2016-02-08,2016,2,8,01:20:00,1,AV BELGRANO Y BERNARDO DE IRIGOYEN,AVENIDA,BELGRANO AV.,True,"BELGRANO AV. e IRIGOYEN, BERNARDO DE",1,Point (107595.35084333 101797.50052813),-58.38048577,-34.61303893,MOTO-CARGAS,MOTO,CARGAS
9,2016-0013,1,2016-02-10,2016,2,10,11:30:00,11,AV ENTRE RIOS 1366,AVENIDA,ENTRE RIOS AV.,False,ENTRE RIOS AV. 1366,1,Point (106616.41069662 100496.44662323),-58.39114932,-34.62477387,PEATON-AUTO,PEATON,AUTO


### Columna ID
Validando que no exista duplicados en la columna ID, columna principal para relacionar con la información del dataframe Víctimas.

In [15]:
hechos_df['ID'].duplicated().sum()

0

### Columnas del tipo fecha (FECHA, AAAA, DD, HORA y HH)
Realizando una revisión en el archivo Excel, se encuentra un caracter en la columna 'HORA' y 'HH' que no corresponde al tipo fecha. Se procede a realizar la actualización del valor a 0, para no perder el resto de información de las columnas relevantes de dicho registro.

In [16]:
hechos_df[hechos_df['HORA'] == 'SD']

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,SD,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


In [17]:
hechos_df['HORA'][518] = 0
hechos_df['HH'][518] = 0

hechos_df.loc[518][['HORA', 'HH', ]]

HORA    0
HH      0
Name: 518, dtype: object

### Columna Dirección Normalizada y Calle
Se había detectado algunos valores nulos en esta columna, considerando que 'SD' son las siglas de 'Sin Datos' se procede a rellenar las columnas vacías con 'SD'.

Empezamos filtrando los valores nulos en la columna 'Dirección Normalizada'.

In [18]:
hechos_df[hechos_df['Dirección Normalizada'].isnull()][['ID', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Dirección Normalizada']]

,ID,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada
38,2016-0052,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN
106,2016-0136,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN
119,2016-0151,SD,CALLE,NaN,NaN
180,2017-0050,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,NaN
181,2017-0051,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NaN
313,2018-0039,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN
546,2020-0026,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN
621,2021-0023,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN


In [19]:
hechos_df['Dirección Normalizada'].fillna('SD', inplace=True)

hechos_df[hechos_df['Dirección Normalizada'] == 'SD'][['ID', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Dirección Normalizada']]

,ID,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada
38,2016-0052,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",SD
106,2016-0136,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,SD
119,2016-0151,SD,CALLE,NaN,SD
180,2017-0050,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,SD
181,2017-0051,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,SD
313,2018-0039,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",SD
546,2020-0026,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",SD
621,2021-0023,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,SD


Realizamos el mismo proceso con la columna 'Calle'

In [20]:
hechos_df[hechos_df['Calle'].isnull()][['ID', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Dirección Normalizada']]

,ID,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada
119,2016-0151,SD,CALLE,NaN,SD


In [21]:
hechos_df['Calle'].fillna('SD', inplace=True)

hechos_df[hechos_df['Calle'] == 'SD'][['ID', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Dirección Normalizada']]

,ID,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada
119,2016-0151,SD,CALLE,SD,SD


Volvemos a revisar que no tengamos valores nulos.

In [22]:
hechos_df.isnull().sum()

ID                       0
N_VICTIMAS               0
FECHA                    0
AAAA                     0
MM                       0
DD                       0
HORA                     0
HH                       0
LUGAR_DEL_HECHO          0
TIPO_DE_CALLE            0
Calle                    0
Cruce                    0
Dirección Normalizada    0
COMUNA                   0
XY (CABA)                0
pos x                    3
pos y                    3
PARTICIPANTES            0
VICTIMA                  0
ACUSADO                  0
dtype: int64

Revisamos los tipos de datos de cada columna

In [23]:
hechos_df.dtypes

ID                               object
N_VICTIMAS                        int64
FECHA                    datetime64[ns]
AAAA                              int64
MM                                int64
DD                                int64
HORA                             object
HH                               object
LUGAR_DEL_HECHO                  object
TIPO_DE_CALLE                    object
Calle                            object
Cruce                              bool
Dirección Normalizada            object
COMUNA                            int64
XY (CABA)                        object
pos x                            object
pos y                            object
PARTICIPANTES                    object
VICTIMA                          object
ACUSADO                          object
dtype: object

Guardamos el Dataset en formato CSV para trabajarlo posteriormente.

In [24]:
hechos_df.to_csv('../Datasets/homicidios_hechos.csv')

## Dataset Victimas
Realizamos una revisión de los datos en el dataset `victimas_df`

In [25]:
victimas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


Hacemos una busqueda global de registros nulos, pero comprobamos que no hay.

In [26]:
victimas_df.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

In [27]:
victimas_df.head(20)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
5,2016-0008,2016-01-24,2016,1,24,CONDUCTOR,MOTO,MASCULINO,30,2016-01-24 00:00:00
6,2016-0009,2016-01-24,2016,1,24,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,29,2016-01-26 00:00:00
7,2016-0010,2016-01-29,2016,1,29,CONDUCTOR,MOTO,MASCULINO,18,2016-01-29 00:00:00
8,2016-0012,2016-02-08,2016,2,8,CONDUCTOR,MOTO,MASCULINO,22,2016-02-08 00:00:00
9,2016-0013,2016-02-10,2016,2,10,PEATON,PEATON,MASCULINO,16,2016-02-10 00:00:00


Revisión de duplicados, esto me permitirá saber si existen registros duplicados en el dataframe de Víctimas.

In [28]:
victimas_df[victimas_df.duplicated()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


En la revisión de columnas, se encontró 68 registros con valores 'SD' en la columna 'FECHA_FALLECIMIENTO', dado que queremos utilizar esta columna, se requiere actualizar los valores a un formato fecha, para estos casos se actualizará con la columna fecha para poder utilizarlo posteriormente en los reportes.

In [29]:
victimas_df.loc[victimas_df['FECHA_FALLECIMIENTO'] == 'SD', 'FECHA_FALLECIMIENTO'] = victimas_df['FECHA']

victimas_df.loc[victimas_df['FECHA_FALLECIMIENTO'] == 'SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


Continuamos revisando el resto de columnas.

In [30]:
victimas_df[victimas_df['ROL']=='SD']


,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,2016-04-17 00:00:00
39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD,2016-04-20 00:00:00
63,2016-0085,2016-06-29,2016,6,29,SD,MOTO,MASCULINO,SD,2016-06-29 00:00:00
77,2016-0101,2016-08-07,2016,8,7,SD,SD,MASCULINO,67,2016-08-07 00:00:00
89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD,2016-09-02 00:00:00
141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,SD,2016-12-27 00:00:00
167,2017-0029,2017-03-07,2017,3,7,SD,SD,MASCULINO,34,2017-03-07 00:00:00
208,2017-0074,2017-06-04,2017,6,4,SD,SD,MASCULINO,70,2017-06-04 00:00:00
221,2017-0089,2017-07-13,2017,7,13,SD,SD,MASCULINO,23,2017-07-13 00:00:00
280,2017-0155,2017-12-12,2017,12,12,SD,SD,MASCULINO,77,2017-12-12 00:00:00


In [31]:
victimas_df[victimas_df['VICTIMA']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,2016-04-17 00:00:00
77,2016-0101,2016-08-07,2016,8,7,SD,SD,MASCULINO,67,2016-08-07 00:00:00
89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD,2016-09-02 00:00:00
93,2016-0119,2016-09-04,2016,9,4,PASAJERO_ACOMPAÑANTE,SD,FEMENINO,SD,2016-09-04 00:00:00
141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,SD,2016-12-27 00:00:00
167,2017-0029,2017-03-07,2017,3,7,SD,SD,MASCULINO,34,2017-03-07 00:00:00
208,2017-0074,2017-06-04,2017,6,4,SD,SD,MASCULINO,70,2017-06-04 00:00:00
221,2017-0089,2017-07-13,2017,7,13,SD,SD,MASCULINO,23,2017-07-13 00:00:00
280,2017-0155,2017-12-12,2017,12,12,SD,SD,MASCULINO,77,2017-12-12 00:00:00


In [32]:
victimas_df[victimas_df['SEXO']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,2016-04-17 00:00:00
39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD,2016-04-20 00:00:00
108,2016-0136,2016-10-25,2016,10,25,CONDUCTOR,MOTO,SD,SD,2016-10-25 00:00:00
121,2016-0151,2016-11-18,2016,11,18,PEATON,PEATON,SD,SD,2016-11-18 00:00:00
138,2016-0171,2016-12-25,2016,12,25,CONDUCTOR,MOTO,SD,SD,2016-12-25 00:00:00
141,2016-0174,2016-12-27,2016,12,27,SD,SD,SD,SD,2016-12-27 00:00:00


En las columnas de ROL, VICTIMA y SEXO, la cantidad de datos sin especificar (SD) es relativamente baja. Específicamente, hay 11 entradas (1.5%) en ROL, 9 entradas (1.3%) en VICTIMA, y 6 entradas (0.8%) en SEXO.

Para evitar dejar estos campos vacíos, se podría reemplazar los valores faltantes en SEXO y ROL con el valor más común encontrado en cada una de estas columnas. Dado que son pocos los datos faltantes, este cambio no debería tener un impacto significativo en los resultados de análisis estadísticos posteriores.

Sin embargo, no modificaré los datos faltantes en la columna VICTIMA para evitar posibles inconsistencias con la información relacionada con las víctimas en el dataframe de Hechos. Cuando se realice la unión (merge) de los dos dataframes, revisaré nuevamente esta columna.

In [33]:
mostCommonGender = victimas_df['SEXO'].mode().iloc[0]
print(f'Para SEXO, el valor que más se repite es: {mostCommonGender}')

Para SEXO, el valor que más se repite es: MASCULINO


In [34]:
victimas_df['SEXO'][victimas_df['SEXO'] == 'SD'] = 'MASCULINO'
victimas_df[victimas_df['SEXO'] == 'SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [35]:
mostCommonRole = victimas_df['ROL'].mode().iloc[0]
print(f'En ROL, el dato más común es: {mostCommonRole}')

En ROL, el dato más común es: CONDUCTOR


In [36]:
victimas_df['ROL'][victimas_df['ROL'] == 'SD'] = 'CONDUCTOR'
victimas_df[victimas_df['ROL'] == 'SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [37]:
victimas_df[victimas_df['EDAD'] == 'SD'].shape[0]

53

La cantidad de datos faltantes en la columna EDAD es considerablemente más alta, aunque sigue representando un porcentaje menor: son 45 registros, lo que equivale a un 6.3%. Mantener estos valores como 'SD' (str) podría causar inconvenientes con el tipo de datos de la columna, y asignarles un valor de 0 podría distorsionar los cálculos estadísticos. Por lo tanto, he decidido utilizar el promedio de edad, diferenciado por el sexo de las víctimas, para completar estos datos faltantes.

Este método reemplaza todos los casos del valor 'SD' en la columna 'EDAD' por pd.NA. Aquí, pd.NA es una forma de representar un valor "No Disponible" o "faltante" en Pandas.

In [38]:
victimas_df['EDAD'] = victimas_df['EDAD'].replace('SD', pd.NA)

promedio_por_genero = victimas_df.groupby('SEXO')['EDAD'].mean()
print(f'La edad promedio de FEMENINO es {round(promedio_por_genero["FEMENINO"])} y de MASCULINO es {round(promedio_por_genero["MASCULINO"])}')

victimas_df['EDAD'] = victimas_df.apply(lambda row: promedio_por_genero[row['SEXO']] if pd.isna(row['EDAD']) else row['EDAD'], axis=1)

victimas_df['EDAD'] = victimas_df['EDAD'].astype(int)

La edad promedio de FEMENINO es 51 y de MASCULINO es 40


In [39]:
victimas_df[victimas_df['EDAD']=='SD']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO


In [40]:
victimas_df.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,2016-01-10 00:00:00
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


Convertimos el tipo object de la columna 'FECHA_FALLECIMIENTO' a tipo datetime.

In [41]:
victimas_df['FECHA_FALLECIMIENTO'] = pd.to_datetime(victimas_df['FECHA_FALLECIMIENTO'])

In [42]:
victimas_df.dtypes

ID_hecho                       object
FECHA                  datetime64[ns]
AAAA                            int64
MM                              int64
DD                              int64
ROL                            object
VICTIMA                        object
SEXO                           object
EDAD                            int32
FECHA_FALLECIMIENTO    datetime64[ns]
dtype: object

In [43]:
victimas_df.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,2016-01-10
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01


Exportamos el dataset a un archivo CSV

In [44]:
victimas_df.to_csv('../Datasets/Homicidios_victimas.csv')

## Merge entre el dataset hechos_df y victimas_df

In [45]:
hechos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  696 non-null    object        
 11  Cruce                  696 non-null    bool          
 12  Dirección Normalizada  696 non-null    object        
 13  COMUN

In [46]:
victimas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    int32         
 9   FECHA_FALLECIMIENTO  717 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int32(1), int64(3), object(4)
memory usage: 53.3+ KB


Dado que las columnas 'AAAA', 'MM', 'DD', 'FECHA' y 'V' se repiten entre los 2 datasets, se procede a eliminarlo las columnas duplicadas.

In [51]:
victimas_df = victimas_df.drop(columns=['FECHA', 'AAAA', 'MM', 'DD'])
hechos_df = hechos_df.drop(columns=['VICTIMA'])


In [48]:
victimas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   ROL                  717 non-null    object        
 2   VICTIMA              717 non-null    object        
 3   SEXO                 717 non-null    object        
 4   EDAD                 717 non-null    int32         
 5   FECHA_FALLECIMIENTO  717 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(4)
memory usage: 30.9+ KB


Realizamos el cambio de nombre de la columna 'ID_hecho' a 'ID'

In [69]:
victimas_df.rename(columns={'ID_hecho': 'ID'}, inplace=True)

In [70]:
victimas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   717 non-null    object        
 1   ROL                  717 non-null    object        
 2   VICTIMA              717 non-null    object        
 3   SEXO                 717 non-null    object        
 4   EDAD                 717 non-null    int32         
 5   FECHA_FALLECIMIENTO  717 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(4)
memory usage: 30.9+ KB


In [71]:
hechos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  696 non-null    object        
 11  Cruce                  696 non-null    bool          
 12  Dirección Normalizada  696 non-null    object        
 13  COMUN

Se realiza el merge entre el dataframe Hechos y Victimas, utilizando la columna ID para unión de los datos.

In [72]:
homicidios_df = pd.merge(hechos_df, victimas_df, on='ID', how='inner')
homicidios_df

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,XY (CABA),pos x,pos y,PARTICIPANTES,ACUSADO,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,PASAJEROS,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,SD,CONDUCTOR,MOTO,MASCULINO,18,2016-01-10
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,PASAJEROS,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18
713,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20
714,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,CARGAS,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02
715,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,CARGAS,CONDUCTOR,AUTO,MASCULINO,60,2021-12-20


In [73]:
homicidios_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     717 non-null    object        
 1   N_VICTIMAS             717 non-null    int64         
 2   FECHA                  717 non-null    datetime64[ns]
 3   AAAA                   717 non-null    int64         
 4   MM                     717 non-null    int64         
 5   DD                     717 non-null    int64         
 6   HORA                   717 non-null    object        
 7   HH                     717 non-null    object        
 8   LUGAR_DEL_HECHO        717 non-null    object        
 9   TIPO_DE_CALLE          717 non-null    object        
 10  Calle                  717 non-null    object        
 11  Cruce                  717 non-null    bool          
 12  Dirección Normalizada  717 non-null    object        
 13  COMUN

In [74]:
homicidios_df[homicidios_df['N_VICTIMAS'] > 1].head(10)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,XY (CABA),pos x,pos y,PARTICIPANTES,ACUSADO,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
29,2016-0041,2,2016-03-29,2016,3,29,11:00:00,11,AV DIRECTORIO Y RIVERA INDARTE,AVENIDA,...,Point (100232.38564985 99530.25843190),-58.46076550,-34.63350444,MOTO-CARGAS,CARGAS,CONDUCTOR,MOTO,MASCULINO,54,2016-03-29
30,2016-0041,2,2016-03-29,2016,3,29,11:00:00,11,AV DIRECTORIO Y RIVERA INDARTE,AVENIDA,...,Point (100232.38564985 99530.25843190),-58.46076550,-34.63350444,MOTO-CARGAS,CARGAS,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,39,2016-03-30
98,2016-0126,2,2016-09-18,2016,9,18,22:45:00,22,IRIGOYEN Y TINOGASTA,CALLE,...,Point (94275.54271123 100886.87954649),-58.52572109,-34.62125906,AUTO-CARGAS,CARGAS,CONDUCTOR,AUTO,MASCULINO,37,2016-09-18
99,2016-0126,2,2016-09-18,2016,9,18,22:45:00,22,IRIGOYEN Y TINOGASTA,CALLE,...,Point (94275.54271123 100886.87954649),-58.52572109,-34.62125906,AUTO-CARGAS,CARGAS,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,60,2016-09-18
163,2017-0026,2,2017-02-26,2017,2,26,05:15:00,5,AV. PERITO MORENO Y FOURNIER,AVENIDA,...,Point (104113.67806500 97722.68219304),-58.41842777,-34.64979057,AUTO-OBJETO FIJO,OBJETO FIJO,PASAJERO_ACOMPAÑANTE,AUTO,FEMENINO,23,2017-02-26
164,2017-0026,2,2017-02-26,2017,2,26,05:15:00,5,AV. PERITO MORENO Y FOURNIER,AVENIDA,...,Point (104113.67806500 97722.68219304),-58.41842777,-34.64979057,AUTO-OBJETO FIJO,OBJETO FIJO,CONDUCTOR,AUTO,MASCULINO,19,2017-02-26
173,2017-0035,3,2017-03-23,2017,3,23,05:00:00,5,AV. DR. TRISTAN ACHAVAL RODRIGUEZ Y BLVD. AZUC...,AVENIDA,...,Point (109583.11620052 102006.72069921),-58.35881506,-34.61113641,AUTO-OBJETO FIJO,OBJETO FIJO,CONDUCTOR,AUTO,MASCULINO,28,2017-03-23
174,2017-0035,3,2017-03-23,2017,3,23,05:00:00,5,AV. DR. TRISTAN ACHAVAL RODRIGUEZ Y BLVD. AZUC...,AVENIDA,...,Point (109583.11620052 102006.72069921),-58.35881506,-34.61113641,AUTO-OBJETO FIJO,OBJETO FIJO,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,32,2017-03-23
175,2017-0035,3,2017-03-23,2017,3,23,05:00:00,5,AV. DR. TRISTAN ACHAVAL RODRIGUEZ Y BLVD. AZUC...,AVENIDA,...,Point (109583.11620052 102006.72069921),-58.35881506,-34.61113641,AUTO-OBJETO FIJO,OBJETO FIJO,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,30,2017-03-23
176,2017-0036,2,2017-03-29,2017,3,29,18:00:00,18,CURUPAYTI Y COLECTORA AV. GRAL. PAZ,GRAL PAZ,...,Point (95300.18060161 105287.23850698),-58.51452347,-34.58159762,MOTO-PASAJEROS,PASAJEROS,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,50,2017-03-29


Exportamos el dataframe a CSV

In [75]:
homicidios_df.to_csv('../Datasets/homicidios_merge.csv')

## Información poblacional del censo CABA 2022
URL: https://censo.gob.ar/index.php/datos_definitivos_caba/

In [82]:
caba2022_df = pd.read_excel('../Datasets/c2022_caba_est_c1_1.xlsx', sheet_name='Editado')
caba2022_df

,Código,Comuna,Población 2010,Población 2022,Variación absoluta,Variación relativa (%),Aumento poblacional (Anual),Población 2016,Población 2017,Población 2018,Población 2019,Población 2020,Población 2021
0,2,Total,2890151,3121707,231556,8.0,19296.333333,3005929.0,3.025225e+06,3.044522e+06,3063818.00,3.083114e+06,3.102411e+06
1,2007,Comuna 1,205886,223554,17668,8.6,1472.333333,214720.0,2.161923e+05,2.176647e+05,219137.00,2.206093e+05,2.220817e+05
2,2014,Comuna 2,157932,161645,3713,2.4,309.416667,159788.5,1.600979e+05,1.604073e+05,160716.75,1.610262e+05,1.613356e+05
3,2021,Comuna 3,187537,196240,8703,4.6,725.250000,191888.5,1.926138e+05,1.933390e+05,194064.25,1.947895e+05,1.955148e+05
4,2028,Comuna 4,218245,229240,10995,5.0,916.250000,223742.5,2.246588e+05,2.255750e+05,226491.25,2.274075e+05,2.283238e+05
5,2035,Comuna 5,179005,194271,15266,8.5,1272.166667,186638.0,1.879102e+05,1.891823e+05,190454.50,1.917267e+05,1.929988e+05
6,2042,Comuna 6,176076,203043,26967,15.3,2247.250000,189559.5,1.918068e+05,1.940540e+05,196301.25,1.985485e+05,2.007958e+05
7,2049,Comuna 7,220591,215896,-4695,-2.1,-391.250000,218243.5,2.178522e+05,2.174610e+05,217069.75,2.166785e+05,2.162872e+05
8,2056,Comuna 8,187237,204367,17130,9.1,1427.500000,195802.0,1.972295e+05,1.986570e+05,200084.50,2.015120e+05,2.029395e+05
9,2063,Comuna 9,161797,169063,7266,4.5,605.500000,165430.0,1.660355e+05,1.666410e+05,167246.50,1.678520e+05,1.684575e+05


In [83]:
caba2022_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Código                       16 non-null     int64  
 1   Comuna                       16 non-null     object 
 2   Población 2010               16 non-null     int64  
 3   Población 2022               16 non-null     int64  
 4   Variación absoluta           16 non-null     int64  
 5   Variación relativa (%)       16 non-null     float64
 6   Aumento poblacional (Anual)  16 non-null     float64
 7   Población 2016               16 non-null     float64
 8   Población 2017               16 non-null     float64
 9   Población 2018               16 non-null     float64
 10  Población 2019               16 non-null     float64
 11  Población 2020               16 non-null     float64
 12  Población 2021               16 non-null     float64
dtypes: float64(8), int64(4

## KPIs

1. Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

*Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales / Población total) * 100,000*

In [84]:
homicidios_df

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,XY (CABA),pos x,pos y,PARTICIPANTES,ACUSADO,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,PASAJEROS,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,SD,CONDUCTOR,MOTO,MASCULINO,18,2016-01-10
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,PASAJEROS,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18
713,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20
714,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,CARGAS,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02
715,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,CARGAS,CONDUCTOR,AUTO,MASCULINO,60,2021-12-20


Calcular la cantidad de muertes correspondientes a cada semestre de los años registrados en el dataframe `homicidios_df`

In [86]:
def determinar_semestre(mes):
    if 1 <= mes <= 6:
        return 1
    elif 7 <= mes <= 12:
        return 2
    else:
        return 'Mes incorrecto'

homicidios_df['Semestre'] = homicidios_df['MM'].apply(determinar_semestre)

In [88]:
homicidios_df.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,pos x,pos y,PARTICIPANTES,ACUSADO,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Semestre
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,-58.47533969,-34.68757022,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,1
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,-58.50877521,-34.66977709,AUTO-PASAJEROS,PASAJEROS,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,1
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,-58.39040293,-34.63189362,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,1
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,-58.46503904,-34.68092974,MOTO-SD,SD,CONDUCTOR,MOTO,MASCULINO,18,2016-01-10,1
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,-58.38718297,-34.62246630,MOTO-PASAJEROS,PASAJEROS,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01,1


Agrupando los registros por año y semestre

In [89]:
victimas_semestre_df = homicidios_df.groupby(['AAAA', 'Semestre'])['N_VICTIMAS'].count().reset_index()
victimas_semestre_df    

,AAAA,Semestre,N_VICTIMAS
0,2016,1,65
1,2016,2,81
2,2017,1,69
3,2017,2,71
4,2018,1,70
5,2018,2,79
6,2019,1,57
7,2019,2,47
8,2020,1,31
9,2020,2,50


In [90]:
victimas_semestre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   AAAA        12 non-null     int64
 1   Semestre    12 non-null     int64
 2   N_VICTIMAS  12 non-null     int64
dtypes: int64(3)
memory usage: 420.0 bytes


Cambiamos de nombre la columna 'AAAA' a 'Año'

In [101]:
victimas_semestre_df.rename(columns={'AAAA': 'Año'}, inplace=True)

victimas_semestre_df

,Año,Semestre,N_VICTIMAS
0,2016,1,65
1,2016,2,81
2,2017,1,69
3,2017,2,71
4,2018,1,70
5,2018,2,79
6,2019,1,57
7,2019,2,47
8,2020,1,31
9,2020,2,50


Necesito obtener los datos correspondientes a la población total de la Ciudad Autónoma de Buenos Aires (CABA) para cada año a partir del DataFrame `caba2022_df`.

In [92]:
#Extraigo la información de población para cada año
poblacion_df= caba2022_df[['Población 2016','Población 2017', 'Población 2018','Población 2019', 
                                  'Población 2020', 'Población 2021']]

poblacion_df

#cambio nombres de las columnas
poblacion_df = poblacion_df.rename(columns={'Población 2016':'2016',
                                                        'Población 2017': '2017', 
                                                        'Población 2018': '2018',
                                                        'Población 2019': '2019', 
                                                        'Población 2020': '2020', 
                                                        'Población 2021': '2021'                  
                                                        })

In [93]:
poblacion_df

,2016,2017,2018,2019,2020,2021
0,3005929.0,3.025225e+06,3.044522e+06,3063818.00,3.083114e+06,3.102411e+06
1,214720.0,2.161923e+05,2.176647e+05,219137.00,2.206093e+05,2.220817e+05
2,159788.5,1.600979e+05,1.604073e+05,160716.75,1.610262e+05,1.613356e+05
3,191888.5,1.926138e+05,1.933390e+05,194064.25,1.947895e+05,1.955148e+05
4,223742.5,2.246588e+05,2.255750e+05,226491.25,2.274075e+05,2.283238e+05
5,186638.0,1.879102e+05,1.891823e+05,190454.50,1.917267e+05,1.929988e+05
6,189559.5,1.918068e+05,1.940540e+05,196301.25,1.985485e+05,2.007958e+05
7,218243.5,2.178522e+05,2.174610e+05,217069.75,2.166785e+05,2.162872e+05
8,195802.0,1.972295e+05,1.986570e+05,200084.50,2.015120e+05,2.029395e+05
9,165430.0,1.660355e+05,1.666410e+05,167246.50,1.678520e+05,1.684575e+05


Selecciono únicamente los datos de la primera fila, que corresponden a los valores totales.

In [94]:
poblacion_df = poblacion_df[:1]
poblacion_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2016    1 non-null      float64
 1   2017    1 non-null      float64
 2   2018    1 non-null      float64
 3   2019    1 non-null      float64
 4   2020    1 non-null      float64
 5   2021    1 non-null      float64
dtypes: float64(6)
memory usage: 180.0 bytes



Reajusto la disposición del DataFrame para obtener el formato deseado de filas y columnas.

In [95]:
poblacion_año_df = pd.melt(poblacion_df, var_name='Año', value_name='Población Total').sort_values(by='Año')


In [97]:
poblacion_año_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Año              6 non-null      object 
 1   Población Total  6 non-null      float64
dtypes: float64(1), object(1)
memory usage: 228.0+ bytes


Modificando el tipo de dato de la columna 'Año' al tipo entero

In [98]:
poblacion_año_df['Año'] = poblacion_año_df['Año'].astype(int)

Unimos ambos dataframes para obtener el KPI 1

In [102]:
kpi1_df = pd.merge(victimas_semestre_df, poblacion_año_df, on='Año', how='left')
kpi1_df

,Año,Semestre,N_VICTIMAS,Población Total
0,2016,1,65,3.005929e+06
1,2016,2,81,3.005929e+06
2,2017,1,69,3.025225e+06
3,2017,2,71,3.025225e+06
4,2018,1,70,3.044522e+06
5,2018,2,79,3.044522e+06
6,2019,1,57,3.063818e+06
7,2019,2,47,3.063818e+06
8,2020,1,31,3.083114e+06
9,2020,2,50,3.083114e+06


Agregando la columna de tasa semestral por cada semestre

In [103]:
kpi1_df['Tasa semestral'] = kpi1_df['N_VICTIMAS']*100000/kpi1_df['Población Total']
kpi1_df

,Año,Semestre,N_VICTIMAS,Población Total,Tasa semestral
0,2016,1,65,3.005929e+06,2.162393
1,2016,2,81,3.005929e+06,2.694674
2,2017,1,69,3.025225e+06,2.280822
3,2017,2,71,3.025225e+06,2.346933
4,2018,1,70,3.044522e+06,2.299212
5,2018,2,79,3.044522e+06,2.594825
6,2019,1,57,3.063818e+06,1.860424
7,2019,2,47,3.063818e+06,1.534034
8,2020,1,31,3.083114e+06,1.005477
9,2020,2,50,3.083114e+06,1.621737


In [104]:
kpi1_df.to_csv('../Datasets/kpi1.csv')

2. Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

*Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100*

In [106]:
moticiclistas_df = homicidios_df[homicidios_df['VICTIMA']=='MOTO']
moticiclistas_df

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,pos x,pos y,PARTICIPANTES,ACUSADO,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Semestre
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,-58.47533969,-34.68757022,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,1
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,-58.39040293,-34.63189362,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,1
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,-58.46503904,-34.68092974,MOTO-SD,SD,CONDUCTOR,MOTO,MASCULINO,18,2016-01-10,1
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,-58.38718297,-34.62246630,MOTO-PASAJEROS,PASAJEROS,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01,1
5,2016-0008,1,2016-01-24,2016,1,24,18:30:00,18,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,...,-58.44451316,-34.68475866,MOTO-OBJETO FIJO,OBJETO FIJO,CONDUCTOR,MOTO,MASCULINO,30,2016-01-24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,2021-0089,1,2021-12-02,2021,12,2,01:10:00,1,AV. GAONA 3655,AVENIDA,...,-58.47633683,-34.62140594,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,41,2021-12-11,2
710,2021-0091,1,2021-12-11,2021,12,11,23:00:00,23,BAIGORRIA Y VICTOR HUGO,CALLE,...,-58.51989389,-34.62284918,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,24,2021-12-11,2
712,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,-58.43353773,-34.64561636,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18,2
713,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,-58.46739825,-34.65117757,MOTO-AUTO,AUTO,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20,2


In [111]:
motos_accidentes_df = moticiclistas_df.groupby('AAAA')['VICTIMA'].count().reset_index()
motos_accidentes_df

,AAAA,VICTIMA
0,2016,65
1,2017,56
2,2018,57
3,2019,50
4,2020,29
5,2021,46


In [113]:
motos_accidentes_df.rename(columns={'AAAA': 'Año'}, inplace=True)
motos_accidentes_df

,Año,VICTIMA
0,2016,65
1,2017,56
2,2018,57
3,2019,50
4,2020,29
5,2021,46


In [109]:
motos_accidentes_df['Año'] = motos_accidentes_df['Año'].astype(int)

In [114]:
kpi2_df = pd.merge(motos_accidentes_df, poblacion_año_df, on='Año', how='left')
kpi2_df

,Año,VICTIMA,Población Total
0,2016,65,3.005929e+06
1,2017,56,3.025225e+06
2,2018,57,3.044522e+06
3,2019,50,3.063818e+06
4,2020,29,3.083114e+06
5,2021,46,3.102411e+06


In [115]:
kpi2_df["Víctimas año anterior"] = kpi2_df["VICTIMA"].shift(periods=1, fill_value=0)
kpi2_df

,Año,VICTIMA,Población Total,Víctimas año anterior
0,2016,65,3.005929e+06,0
1,2017,56,3.025225e+06,65
2,2018,57,3.044522e+06,56
3,2019,50,3.063818e+06,57
4,2020,29,3.083114e+06,50
5,2021,46,3.102411e+06,29


In [116]:
kpi2_df['Evolución']= (kpi2_df['Víctimas año anterior']-kpi2_df['VICTIMA'])*100/kpi2_df['Víctimas año anterior']
kpi2_df

,Año,VICTIMA,Población Total,Víctimas año anterior,Evolución
0,2016,65,3.005929e+06,0,-inf
1,2017,56,3.025225e+06,65,13.846154
2,2018,57,3.044522e+06,56,-1.785714
3,2019,50,3.063818e+06,57,12.280702
4,2020,29,3.083114e+06,50,42.000000
5,2021,46,3.102411e+06,29,-58.620690


In [117]:
kpi2_df = kpi2_df.rename(columns={'VICTIMA':'Cantidad de víctimas' })

In [118]:
kpi2_df.to_csv('../Datasets/kpi2.csv')